In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("tormarket")

create_database('blp_dataset')
create_table('products_tmp_2')

In [ ]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\products")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            
            if page_content is not None:
                ## GET DESCRIPTION
                description = get_content(page_content.find("pre"))

                ## GET TITLE
                raw_title = page_content.find("div", attrs={"id": "product-title"})
                title = get_content(raw_title)

                ## GET SELLER
                seller = ""
                raw_seller = page_content.find_all("a", href=lambda href: href and "/users/" in href)

                if raw_seller:
                    seller = get_content(raw_seller[0])

                
                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_2 (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "Tor Market", seller, "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid
                    
                    ## GET IMAGE
                    img_content = page_content.find_all("img", src=lambda src: src and "/uploaded/" in src)
                    name = ""
                    
                    if img_content:
                        raw_name = img_content[0]
                        
                        if raw_name is not None:
                            try:
                                name = raw_name["src"]
                            except IndexError:
                                print(f"Image {product_id} has no src")
                                update_image(product_id, "no_image")

                        if name is not "":
                            name = name.replace("?", "_")
                            url_image = Path(f"{folder_image}\{name}")
                            ver = 0

                            if not url_image.exists():
                                name = name.replace("thumb", "medium")
                                url_image = Path(f"{folder_image}\{name}")
                                if url_image.exists(): ver = 1
                                
                            if not url_image.exists() and ver == 0:
                                name = name.replace("thumb", "original")
                                url_image = Path(f"{folder_image}\{name}")
                                if url_image.exists(): ver = 1
                            
                            if not url_image.exists() and ver == 0:
                                    name = name.replace("medium", "thumb")
                                    url_image = Path(f"{folder_image}\{name}")
                                    if url_image.exists(): ver = 1
                            
                            if not url_image.exists() and ver == 0:
                                    name = name.replace("medium", "original")
                                    url_image = Path(f"{folder_image}\{name}")
                                    if url_image.exists(): ver = 1
                            
                            if not url_image.exists() and ver == 0:
                                    name = name.replace("original", "thumb")
                                    url_image = Path(f"{folder_image}\{name}")
                                    if url_image.exists(): ver = 1
                            
                            if not url_image.exists() and ver == 0:
                                    name = name.replace("original", "medium")
                                    url_image = Path(f"{folder_image}\{name}")
                                    if url_image.exists(): ver = 1

                            if url_image.exists():
                                try:
                                    # copy images to new directory, renaming with product id
                                    if url_image.exists():
                                        print(f"Image {product_id} exists... creating")

                                        dst_dir = Path(f"D:\images\p\{product_id}.jpg")
                                        copyfile(url_image, dst_dir)
                                    else:
                                        print(f"Image {product_id} does not exists")
                                        update_image(product_id, "no_image")
                                except:
                                    print(f"Image {product_id} has error. Except")
                                    update_image(product_id, "no_image")
                            else:
                                print(f"Image {product_id} does not exists")
                                update_image(product_id, "no_image")
                        else:
                            print(f"Image {product_id} no exists")
                            update_image(product_id, "no_image")
                        
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image")